In [ ]:
# Grab the data from github
# These csv files have been pre-processed / cleaned to make it easier to combine, plot etc

import pandas as pd
url = "https://github.com/calcut/gauldry-lab-data/raw/main/lab_data.csv"
lab_df = pd.read_csv(url)


url = "https://github.com/calcut/gauldry-lab-data/raw/main/sensor_data_cleaned.csv"
sensor_df = pd.read_csv(url)

# combine the two dataframes
full_df = pd.concat([sensor_df, lab_df], axis=0, ignore_index=True)

# Keep only the columns we need
full_df = full_df[["timestamp", "value", "tank", "determinand", "unit"]]

# display a summary of the data
full_df

In [ ]:
# Define the colours and labels for the plots

color_mapping = {
    "30C.1": "#c586c0",
    "30C.2": "#c586c0",
    "30C.3": "#c586c0",
    "20C.1": "#ce9178",
    "20C.2": "#ce9178",
    "20C.3": "#ce9178",
    "INS.1": "#dcdcaa",
    "INS.2": "#dcdcaa",
    "INS.3": "#dcdcaa",
    "CON.1": "#9cdcfe",
    "CON.2": "#9cdcfe",
    "CON.3": "#9cdcfe",
    "AMB"  : "#5edf7e",
    "INLET": "#8e3030",
}

labels_mapping = {
    "ts"                            : "Top Temp (C)",
    "tl"                            : "Bottom Temp (C)",
    "ph"                            : "pH",
    "pr"                            : "Pressure (mBar)",
    "gc"                            : "CH4 Concentration (%)",
    'Ammonia'                       : 'Ammonia (mg/l as N)',
    'Nitrate'                       : 'Nitrate (mg/l as N)',
    'Nitrite'                       : 'Nitrite (mg/l as N)',
    'Phosphorus'                    : 'Phosphorus (mgP/l)',
    'Soluble Chemical Oxygen Demand': 'Soluble Chemical Oxygen Demand (mgO₂/l)',
    'Sulphate'                      : 'Sulphate (mgSO₄/l)',
    'Suspended solids'              : 'Suspended solids (mg/l)',
    'Total Chemical Oxygen Demand'  : 'Total Chemical Oxygen Demand (mgO₂/l)',
    'Total Nitrogen (as N)'         : 'Total Nitrogen (as N) (mg/l as N)',
    'Total oxidised nitrogen'       : 'Total oxidised nitrogen (mg/l as N)',
    'Alkalinity'                    : 'Alkalinity (mgCaCO₃/l)',
    'Biochemical oxygen demand'     : 'Biochemical oxygen demand (mgO₂/l)',
    'Conductivity'                  : 'Conductivity (mS/cm)',
    'Manganese'                     : 'Manganese (mgMn/l)',
    'Total organic carbon'          : 'Total organic carbon (mgC/l)',
}

In [ ]:
# Examples of how to filter data, apply as many filters as you want

selected_data = full_df

# select lab data from a date range
selected_data = selected_data[(selected_data['timestamp'] >= '2023-04-01') & (selected_data['timestamp'] <= '2023-04-10')]

# filter for only ammonia and nitrate data and the short thermocouple
selected_data = selected_data[selected_data['determinand'].isin(['Ammonia', 'Nitrate', 'ts'])]

# filter for only tank names starting with 30C
selected_data = selected_data[selected_data['tank'].str.startswith('30C')]

# check what data remains
selected_data

In [ ]:
import plotly.express as px
import plotly.io as pio

fig = px.line(selected_data,
            x="timestamp",
            y="value",
            title='Gauldry Sensor Data',
            color="tank",
            custom_data=["value"],
            color_discrete_map=color_mapping,
            facet_row="determinand",
            facet_row_spacing=0.015,
            height=900,
            # width=1000,
            )

# customise hover label to just show the value
fig.update_traces(
    hovertemplate="<br>".join([
        "%{customdata[0]}",
    ])
)

# Show all traces on the hover label
fig.update_layout(hovermode="x")

# prevent the Y axis from being shared between the subplots
fig.update_yaxes(matches=None)

# remove the "Determinand=" from Facet Labels
fig.for_each_annotation(lambda a: a.update(text=labels_mapping[a.text.split("=")[-1]]))

# Remove the y axis labels (which was just "value" for all of them)
fig.for_each_yaxis(lambda a: a.update(title=""))

# Show date labels
fig.update_xaxes(title="", showticklabels=True)
                

fig.show()